In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([transforms.ToTensor, 
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [5]:
trainset = torchvision.datasets.FashionMNIST('./fashionMNIST', train = True, 
                                             transform = transform, download = True)
traindata = torch.utils.data.DataLoader(trainset, batch_size = 10, shuffle = True)

testset = torchvision.datasets.FashionMNIST('./fashionMNIST', train = False,
                                           transform = transform, download = True)
testdata = torch.utils.data.DataLoader(testset, batch_size = 10, shuffle = True)

In [6]:
classes = ('T-Shirt/Top', 'Trouser', 'Pullover', 'Dress',
           'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow():
    